In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [ ]:
pip install mlflow dagshub

In [ ]:
import dagshub
dagshub.init(repo_owner='tvani2', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

# Cleaning and engineering

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class FullPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 target_column='isFraud',
                 transaction_thresh=0.6,
                 identity_thresh=0.9,
                 identity_df=None):
        self.target_column = target_column
        self.transaction_thresh = transaction_thresh
        self.identity_thresh = identity_thresh
        self.identity_df = identity_df  # identity will be passed during initialization

    def fit(self, X, y=None):
        # 1. Drop columns with too many missing values
        self.transaction_cols_to_keep = X.columns[X.isnull().mean() < self.transaction_thresh].tolist()
        if self.identity_df is not None:
            self.identity_cols_to_keep = self.identity_df.columns[self.identity_df.isnull().mean() < self.identity_thresh].tolist()
        else:
            self.identity_cols_to_keep = []

        # 2. Merge
        if self.identity_df is not None:
            identity_filtered = self.identity_df[self.identity_cols_to_keep]
            X = X[self.transaction_cols_to_keep].merge(identity_filtered, how='left', on='TransactionID')
        else:
            X = X[self.transaction_cols_to_keep]

        # 3. Separate numeric and categorical columns
        self.numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        self.categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

        # 4. Imputers
        self.numeric_imputer = SimpleImputer(strategy='mean')
        self.categorical_imputer = SimpleImputer(strategy='most_frequent')

        # Fit imputers
        self.numeric_imputer.fit(X[self.numeric_cols])
        self.categorical_imputer.fit(X[self.categorical_cols])

        # 5. Determine WOE and one-hot columns
        s = X[self.categorical_cols].nunique()
        self.woe_columns = list(s[s > 3].index)
        self.one_hot_columns = list(s[s <= 3].index)

        # 6. Fit WOE mappings
        if y is not None:
            df_woe = X[self.woe_columns].copy()
            df_woe['target'] = y.reset_index(drop=True)

            self.woe_mappings = {}
            self.woe_columns_fillna = df_woe[self.woe_columns].mode().T[0].to_dict()

            for col in self.woe_columns:
                groups = df_woe.groupby(col)['target'].agg(['count', 'mean'])
                groups['n_pos'] = groups['mean'] * groups['count']
                groups['n_neg'] = groups['count'] - groups['n_pos']

                total_pos = groups['n_pos'].sum()
                total_neg = groups['n_neg'].sum()

                groups['prop_pos'] = groups['n_pos'] / total_pos
                groups['prop_neg'] = groups['n_neg'] / total_neg

                groups['woe'] = np.log(groups['prop_pos'] / groups['prop_neg'])

                groups.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
                self.woe_mappings[col] = groups['woe'].to_dict()

        return self

    def transform(self, X):
        # 1. Drop columns with too many missing values
        if self.identity_df is not None:
            identity_filtered = self.identity_df[self.identity_cols_to_keep]
            X = X[self.transaction_cols_to_keep].merge(identity_filtered, how='left', on='TransactionID')
        else:
            X = X[self.transaction_cols_to_keep]

        # 2. Impute missing values
        X[self.numeric_cols] = self.numeric_imputer.transform(X[self.numeric_cols])
        X[self.categorical_cols] = self.categorical_imputer.transform(X[self.categorical_cols])

        # 3. Apply WOE encoding
        for col in self.woe_columns:
            new_col = f'{col}_woe'
            X[new_col] = (
                X[col]
                .map(self.woe_mappings[col])
                .fillna(self.woe_mappings[col].get(self.woe_columns_fillna[col], 0))
            )

        # 4. One-hot encode
        X = pd.get_dummies(X, columns=self.one_hot_columns, drop_first=True, dummy_na=True)

        # 5. Drop original WOE and one-hot columns
        cols_to_drop = [col for col in (self.woe_columns + self.one_hot_columns) if col in X.columns]
        X = X.drop(columns=cols_to_drop)

        return X

In [ ]:
from sklearn.impute import SimpleImputer

# Load data
transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')

# Separate target
target_column = 'isFraud'
y = transaction[target_column]
X = transaction.drop(columns=[target_column])

# Initialize preprocessor
preprocessor = FullPreprocessor(
    target_column=target_column,
    transaction_thresh=0.6,
    identity_thresh=0.9,
    identity_df=identity
)

# Fit-transform
X_processed = preprocessor.fit_transform(X, y)

print(X_processed.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Feature selection

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class CorrelationDropper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.9):
        self.threshold = threshold
        self.to_drop_ = None

    def fit(self, X, y=None):
        # 1. Calculate correlation matrix
        corr_matrix = X.corr().abs()
        
        # 2. Upper triangle of the correlation matrix
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        
        # 3. Find features with correlation greater than threshold
        self.to_drop_ = [column for column in upper.columns if any(upper[column] > self.threshold)]
        
        print(f"Columns to drop due to high correlation ({len(self.to_drop_)}): {self.to_drop_}")
        
        return self

    def transform(self, X):
        # 4. Drop them
        X_dropped = X.drop(columns=self.to_drop_, errors='ignore')
        return X_dropped

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [ ]:
dropper = CorrelationDropper(threshold=0.9)

# Fit on training data and transform it
X_train_new = dropper.fit_transform(X_train)

# Only transform test data
X_test_new = dropper.transform(X_test)

# Done!
print("Train set:", X_train_new.shape)
print("Test set:", X_test_new.shape)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def build_random_forest_pipeline():
    """
    Faster Random Forest pipeline with minimal GridSearchCV.
    """
    pipeline = Pipeline([
        ('model', RandomForestClassifier(random_state=42, n_jobs=-1))  # Removed class_weight
    ])

    # Minimal hyperparameter grid
    param_grid = {
        'model__n_estimators': [30],                # Only 1 value
        'model__max_depth': [3, 7],                 # Keep shallow trees
        'model__max_features': ['sqrt']             # Only sqrt (faster)
    }

    # Faster GridSearch (2-fold CV)
    model = GridSearchCV(
        pipeline,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=2,                                      # Fewer folds
        verbose=1,
        n_jobs=-1
    )

    return model

In [ ]:
# 1. Build the model
model = build_random_forest_pipeline()

# 2. Train the model
model.fit(X_train, y_train)

# 3. Predict
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

train_probs = model.predict_proba(X_train)[:, 1]
test_probs = model.predict_proba(X_test)[:, 1]

# 4. Evaluate
from sklearn.metrics import roc_auc_score, classification_report

print("Best parameters found:", model.best_params_)

print("Train ROC-AUC:", roc_auc_score(y_train, train_probs))
print("Test ROC-AUC:", roc_auc_score(y_test, test_probs))

print("\nTrain classification report:\n", classification_report(y_train, train_preds))
print("\nTest classification report:\n", classification_report(y_test, test_preds))

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

def build_random_forest_pipeline():
    """
    Faster Random Forest pipeline with minimal GridSearchCV.
    """
    pipeline = Pipeline([
        ('model', RandomForestClassifier(random_state=42, n_jobs=-1))
    ])

    # Minimal hyperparameter grid
    param_grid = {
        'model__n_estimators': [30],
        'model__max_depth': [3, 7],
        'model__max_features': ['sqrt']
    }

    model = GridSearchCV(
        pipeline,
        param_grid=param_grid,
        scoring='roc_auc',
        cv=2,
        verbose=1,
        n_jobs=-1
    )

    return model

# 1. Build the model
model = build_random_forest_pipeline()

# 2. Start MLflow run
mlflow.set_experiment("Random_Forest_Training")

with mlflow.start_run(run_name="Random_Forest_Run"):
    # 3. Train the model
    model.fit(X_train, y_train)

    # 4. Predict
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)

    train_probs = model.predict_proba(X_train)[:, 1]
    test_probs = model.predict_proba(X_test)[:, 1]

    # 5. Evaluate
    train_roc_auc = roc_auc_score(y_train, train_probs)
    test_roc_auc = roc_auc_score(y_test, test_probs)

    print("Best parameters found:", model.best_params_)

    print("Train ROC-AUC:", train_roc_auc)
    print("Test ROC-AUC:", test_roc_auc)

    print("\nTrain classification report:\n", classification_report(y_train, train_preds))
    print("\nTest classification report:\n", classification_report(y_test, test_preds))

    # 6. Log params, metrics, and model
    mlflow.log_params(model.best_params_)
    mlflow.log_metric("train_roc_auc", train_roc_auc)
    mlflow.log_metric("test_roc_auc", test_roc_auc)

    # Optional: confusion matrix metrics
    train_cm = confusion_matrix(y_train, train_preds)
    test_cm = confusion_matrix(y_test, test_preds)
    
    mlflow.log_metric("train_TN", train_cm[0, 0])
    mlflow.log_metric("train_FP", train_cm[0, 1])
    mlflow.log_metric("train_FN", train_cm[1, 0])
    mlflow.log_metric("train_TP", train_cm[1, 1])
    
    mlflow.log_metric("test_TN", test_cm[0, 0])
    mlflow.log_metric("test_FP", test_cm[0, 1])
    mlflow.log_metric("test_FN", test_cm[1, 0])
    mlflow.log_metric("test_TP", test_cm[1, 1])

    # Save the best model itself
    mlflow.sklearn.log_model(model.best_estimator_, "model")